<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка,-изучение-и-предобработка-данных" data-toc-modified-id="Загрузка,-изучение-и-предобработка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка, изучение и предобработка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Защита персональных данных клиентов

В страховой компании «Хоть потоп» нужно защитить данные клиентов. В проекте необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При этом следует обосновать корректность работы метода преобразования. Защитить персональную информацию нужно так, чтобы при преобразовании качество модели машинного обучения (линейной регрессии) не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка, изучение и предобработка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Загрузим данные из файла и ознакомимся с таблицей.

In [2]:
try:
    main_data = pd.read_csv('insurance.csv')
except:
    main_data = pd.read_csv('/datasets/insurance.csv')
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
main_data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Пропуски в таблице отсутствуют, типы данных верные. Имена столбцов указаны кириллицей и не имеют "змеиного" регистра, но в данной задаче не будем их изменять, так как в этом нет необходимости.

Теперь посмотрим на таблицу с точки зрения описательной статистики.

In [4]:
main_data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Аномальных значений и выбросов нет, что с точки зрения здравого смысла подтверждается по максимальным и минимальным значениям.

Теперь проверим таблицу на наличие строк дубликатов.

In [5]:
main_data.duplicated().sum()

153

В таблице имеются дубликаты, но удалять их не будем. Причина в том, что в таблице нет столбца, который мог бы мог однозначно идентифицировать клиентов (к примеру id). А значит найденные дубликаты могут быть вполне реальными данными, а не лишними копиями строк.

Проверим теперь признаки, которые будут использоваться для обучения модели, на мультиколлинеарность (наличие линейной зависимости). Для этого построим матрицу корреляции. Через настройки визуализации pandas изменим внешний вид итоговой таблицы корреляции.

In [6]:
corr = main_data.loc[:,'Пол':'Члены семьи'].corr()
corr.style.background_gradient(cmap='coolwarm')

,Пол,Возраст,Зарплата,Члены семьи
Пол,1.000000,0.002074,0.014910,-0.008991
Возраст,0.002074,1.000000,-0.019093,-0.006692
Зарплата,0.014910,-0.019093,1.000000,-0.030296
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000


Каких-либо зависимостей между обучающими признаками не выявлено.

На этом предобработка и изучение данных завершены.

## Умножение матриц

**Вопрос:** если матрицу признаков умножить на какую-либо обратимую матрицу, то изменится ли качество линейной регрессии?

**Ответ:** качество линейной регрессии не изменится.

**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Формула вычисления вектора предсказаний при использовании линейной регрессии:

$$
a = Xw
$$

где $w$ определяется решением задачи обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Результирующая формула обучения для вектора $w$:

$$
w = (X^T X)^{-1} X^T y
$$

Полная запись формулы вычисления вектора предсказаний:

$$
a = X (X^T X)^{-1} X^T y 
$$

Умножим исходную матрицу признаков на **квадратную обратимую** матрицу $P$. Размер матрицы $P$ (количество строк или столбцов) будет равен числу столбцов матрицы $X$. Получим преобразованную матрицу признаков  $X'$:

$$ X' = XP $$

Подставим значение этой матрицы в полную запись формулы вычисления предсказаний и получим вектор предсказаний $a'$:

$$ a' = XP ((XP)^T XP)^{-1} (XP)^T y $$

Обратимся к свойствам обратных, транспонированных и единичных матриц:

<ol>
    <li> $(A B)^{-1} =B^{-1} A^{-1}$ ;</li>
    <li>$(A B)^{T} =B^{T} A^{T}$ ;</li>
    <li>$A (BС) =(AB)C$ ;</li>
    <li>$A E=E A=A$ ;</li>
    <li>$A A^{-1}=E$ ;</li>
</ol>
где $E$ - единичная матрица.

Важно учесть, что свойство №1 применимо только для квадратных матриц. 

Применим свойство №2 для вектора предсказаний $a'$:

$ a' = XP (P^TX^TXP)^{-1} P^TX^T y $

Применим свойства №3 и №1, учитывая тот факт, что матрицы $(P^TX^TX)$ и $P$ являются квадратными:

$ a' = XP P^{-1}(P^TX^TX)^{-1} P^TX^T y $

Снова воспользуемся свойством №1 для квадратных матриц $P^T$ и $(X^TX)$:

$ a' = XP P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^T y $

Последовательно применим свойство №5, затем свойство №4:

$ a' = XE(X^TX)^{-1}EX^T y $

$ a' = X(X^TX)^{-1}X^T y $

Полученное выражение полностью соответствует выражению для вектора предсказаний $a$:

$$a' = a$$

Следовательно, умножение матрицы признаков на обратимую матрицу никак не влияет на качество линейной регрессии, ч. т. д.

## Алгоритм преобразования

**Описание алгоритма:** 
<ol>
    <li>сгенерировать случайную квадратную матрицу размером $n*n$, где $n$ - количество столбцов матрицы признаков. Случайные значения матрицы получить на основе равномерного распределения;
    <li>проверить полученную матрицу на обратимость. Для этого проверить, равен ли нулю её определитель. Если равен, то вернуться к шагу 1, если не равен, то перейти к шагу 3;
    <li>умножить исходную матрицу признаков на сгенерированную матрицу. Должна получиться преобразованная матрица признаков;
    <li> создать модель линейной регрессии и обучить её на преобразованной матрице признаков;
    <li> получить предсказания модели (на основе преобразованных признаков) и рассчитать метрику R2;
    <li> заново обучить модель линейной регрессии на исходной матрице признаков без преобразования;
    <li> получить предсказания модели (на основе исходных признаков без преобразования) и рассчитать метрику R2; 
    <li> сравнить значения метрик R2, полученные на шагах 5 и 7.
</ol>
    
**Обоснование алгоритма:** обоснование приведено в п. 2 проекта. Отметим, что в п. 2 использовалась сокращённая запись формулы вычисления предсказаний, и подразумевалось, что исходная матрица признаков содержит также нулевой столбец с единицами. В данном случае предлагается в качестве исходной матрицы признаков использовать матрицу без нулевого столбца (т.е. количество столбцов в матрице будет меньше на единицу). Это действие никак не опровергает доказательства из п. 2, так как обратимая матрица для умножения в алгоритме тоже будет иметь размерность на единицу меньше. При этом все математические выкладки, приведённые в п. 2 будут верны.

## Проверка алгоритма

Для проверки алгоритма следует создать и обучить модель линейной регрессии сначала на матрице признаков без преобразования, а затем на преобразованной матрице. И сравнить результаты качества предсказаний модели. Для оценки качества будем использовать метрику R2 (коэффициент детерминации).

Для начала подготовим обычные и целевые признаки для обучения. Использовать данные с типом np.array необязательно, для обучения модели линейной регрессии подходят и данные типа Dataframe или Series. Матрицы и вектора из них будут получены автоматически в самой модели.

In [7]:
features = main_data.drop('Страховые выплаты', axis=1)
target = main_data['Страховые выплаты']
print(features.shape)
print(target.shape)

(5000, 4)
(5000,)


Обычные и целевые признаки подготовлены. Обратим внимание, что всего у объектов имеется 4 признака, которые будут использоваться в обучении. Именно такого размера будет создаваться квадратная матрица для преобразования.

Теперь создадим непосредственно матрицу, которая будет использована в алгоритме преобразования. Для решения этой задачи используем функцию random.uniform библиотеки numpy, которая создаст матрицу нужного размера и заполнит её случайными значениями на основе равномерного распределения (по умолчанию это числа от 0 до 1). После создания матрицы необходимо убедиться, что она точно является обратимой. Для этого проверим, не является ли она вырожденной (её определитель будет равен нулю). Если вдруг равен, то сгенерируем матрицу заново.

In [8]:
P = 0
det = 0
while det==0:
    P = np.random.uniform(size=(4,4))
    det = np.linalg.det(P)
print(P)

[[0.3070947  0.42771067 0.84936632 0.5064532 ]
 [0.23508564 0.28324128 0.63894454 0.43594383]
 [0.752579   0.82525619 0.75530689 0.45088223]
 [0.80193374 0.41150981 0.7295975  0.08875838]]


Матрица успешно создана. Теперь выполним преобразование исходных признаков по предложенному алгоритму (умножим исходные признаки на созданную матрицу). Можно не обращаться к атрибуту values у объекта Dataframe. Корректное умножение матриц произойдёт автоматически. В результате объект будет по-прежнему иметь тип Dataframe.

In [9]:
features_secured = features @ P
features_secured.head()

,0,1,2,3
0,37338.665968,40945.159368,37490.997479,22382.227284
1,28609.617895,31373.176005,28731.782901,17153.666736
2,15810.976495,17338.594085,15879.974101,9481.169102
3,31389.084989,34419.954403,31511.174381,18811.121132
4,19649.201407,21547.545147,19732.249666,11780.738961


Преобразование исходных признаков прошло успешно. Разобрать персональную информацию клиентов без дешифровки не представляется возможным.

Теперь создадим модель линейной регресиии, попробуем сначала обучить её на исходных данных без преобразования и получить предсказания. Посчитаем метрику R2. Затем заново обучим модель, но уже на преобразованных данных. Сравним результаты.

In [10]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_result = r2_score(target, predictions)
print(f"{'Значение метрики R2 на основе данных без преобразования: ':<60}{r2_result}")

model.fit(features_secured, target)
predictions = model.predict(features_secured)
r2_result = r2_score(target, predictions)
print(f"{'Значение метрики R2 на основе преобразованных данных: ':<60}{r2_result}")

Значение метрики R2 на основе данных без преобразования:    0.42494550286668
Значение метрики R2 на основе преобразованных данных:       0.4249455028666801


Значения метрик R2 практически идентичны. Различие проявляется лишь на уровне погрешности. Таким образом, правильность предложенного алгоритма преобразования успешно проверена опытным путём.

## Общий вывод

В проекте был исследован вопрос о влиянии преобразования исходной матрицы признаков на качество линейной регрессии. В качестве преобразования рассматривалось умножение на какую-либо обратимую матрицу. С помощью математических выкладок и преобразований формулы вычисления предсказаний линейной регрессии было доказано, что умножение матрицы признаков на обратимую квадратную матрицу никак не влияет на качество линейной регрессии.

Далее был предложен конкретный алгоритм преобразования - умножение исходной матрицы признаков на квадратную обратимую матрицу размерностью $n*n$, где $n$ - количество столбцов матрицы признаков. В качестве квадратной матрицы предложено использовать матрицу со случайными значениями на основе равномерного распределения. В исходных данных после преобразования невозможно определить персональную информацию о клиентах без дешифровки.

Алгоритм преобразования затем был успешно проверен на работе реальной модели линейной регрессии. Сначала модель обучалась на матрице признаков без преобразования, а затем на преобразованной матрице. По предсказаниям модели в первом и втором случаях были посчитаны метрики качества R2, которые оказались идентичными.